In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls drive/MyDrive/Data

1data.csv  cmt_data1  url_40k.csv


In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import math
import threading
import time
import concurrent.futures
from tqdm import tqdm


### Preapre header , params, root_link and data.

In [ ]:
root_url = 'https://www.foody.vn/__get/Review/ResLoadMore'

params = {
    't':'',
    'ResId':'6644',
    'Count':'10',
    'Type':'1',
    'fromOwner':'',
    'isLatest':'true',
    'ExcludeIds':'',
}

root_refer = 'https://www.foody.vn/ho-chi-minh/'

headers = {
    'Accept':'application/json, text/plain, */*',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection':'keep-alive',
    'Cookie':'',
    'Host':'www.foody.vn',
    'Referer':'',
    'Sec-Fetch-Dest':'empty',
    'Sec-Fetch-Mode':'cors',
    'Sec-Fetch-Site':'same-origin',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:107.0) Gecko/20100101 Firefox/107.0',
    'X-Requested-With':'XMLHttpRequest',
}

file contains restaurant_id (restaurant_url and restuarant_name are uesd for testing purpose)

In [ ]:
url_df = pd.read_csv(r'drive/MyDrive/Data/url_40k.csv')
url_df

,restaurant_id,restaurant_url,restaurant_name
0,786442,lien-hoa-ga-ac-oc-heo-tiem-thuoc-bac,Liên Hoa - Gà Ác & Óc Heo Tiềm Thuốc Bắc
1,14,ngoc-suong-ben-thuyen-nguyen-van-troi,NS Bến Thuyền
2,786494,bun-mi-vang-hu-tieu-mi-thap-cam,Bún Mì Vàng & Hủ Tiếu Mì Thập Cẩm
3,73,hai-san-be-vang,Hải Sản Bê Vàng
4,786523,sinh-to-minh-thien-le-van-luong,Sinh Tố Minh Thiên - Lê Văn Lương
...,...,...,...
41762,1109293,bep-nha-my-cappuchino-dua-ca-phe-muoi-duong-so-7,Bếp Nhà Mỳ - Cappuchino Dừa & Cà Phê Muối - Đư...
41763,1000027207,me-nau-tra-dao-tra-trai-cay-trinh-dinh-trong.m...,"Mẹ Nấu - Trà Đào, Trà Trái Cây - Trịnh Đình Trọng"
41764,1144332,coolabah-saigon-happy-valley,Coolabah - Bistro - Nguyễn Văn Linh
41765,1000024723,tap-hoa-bo-beo-an-vat-giai-khat-ly-thai-to.uzsetz,Tạp Hóa Bơ Béo - Ăn Vặt & Giải Khát - Lý Thái Tổ


### Crawl

In [ ]:
def crawl_cmt2(id, params):
    cmt_data = []

    params['ResId'] = id
    params['ExcludeIds'] = ''

    # print(params)

    re = requests.get(root_url, params = params, headers = headers)
    # print(re.url)

    content = re.json()
    cmts = content['Items']

    counter = 0
    old_cmt_num = 0

    while len(cmts):
        # print('counter:  ', counter)
        counter+=1

        tempdf = pd.json_normalize(cmts)
        cmt_data.append(tempdf)

        temp_excludeIds = tempdf.Id.tolist()
        params['ExcludeIds'] += ','+ str(temp_excludeIds).replace(']','').replace('[','')
        params['ExcludeIds'] = params['ExcludeIds'].strip(',')


        temp_ls = params['ExcludeIds']
        temp_list = list(set(temp_ls.split(',')))
        params['ExcludeIds'] = ','.join(temp_list).strip(',')

        # print(params['ExcludeIds'])
        # print('len params:   ',len(temp_list))

        new_cmt_num = len(temp_list)
        
        if old_cmt_num == new_cmt_num:
            break
        else:
            old_cmt_num = new_cmt_num

        re = requests.get(root_url, params = params, headers = headers)
        cmts = re.json()['Items']
        # print('len cmts:   ', len(cmts))



    # print("len cmt_data: ", len(cmt_data))

    if len(cmt_data):
        data = pd.concat(cmt_data, ignore_index = True)
        data['Restaurant_id'] = content['ResId']
        return data
    return []

- Because Foody restrict the number of crawling at the time at about 6k requests. 
- So we just crawl 4k for one runtime. 
- Then delete runtime and run again(diconnect and delete runtime, not restart and run all)

In [ ]:
ii = 0 # this is a number of runtime.

In [ ]:
data = []
fail_ids = []
total_fail = 0
total_success = 0

max_workers = 1000

num = 4000

begin = num* ii
length1 = begin + num
length2 = len(url_df)
length = min(length1, length2)

pbar = tqdm(total = (length - begin))

with concurrent.futures.ThreadPoolExecutor(max_workers= max_workers) as executor:
    future_samples = []
    map_pool = {}
    index = begin

    while index < length:
        future_samples = []

        for i in range(max_workers):
            id = url_df.loc[index, 'restaurant_id']
            sample = executor.submit(crawl_cmt2, id , params.copy())
            future_samples.append(sample)
            map_pool[sample] = id

            index += 1
            if index >= length:
                break

        for future in concurrent.futures.as_completed(future_samples):
            try:
                tempdata = future.result()
                if len(tempdata):
                    data.append(tempdata)
                total_success += 1
            except Exception as exc:
                total_fail += 1
                fail_ids.append(map_pool[future])
                print('Generated an exception: %s' % (exc))
                print(map_pool[future])
                print()
            pbar.update(1)       

 99%|█████████▉| 3967/4000 [03:02<00:00, 65.50it/s]

In [ ]:
print('fail : ', total_fail)
print('fail id: ', len(fail_ids)) 
print('success: ', total_success)
print('total:  ', total_fail + total_success)

fail :  0
fail id:  0
success:  4000
total:   4000


#### test fail id

These blocks of code if for testing purpose if there are any fail requests.

In [ ]:
# index_n = 1000

# id = url_df.loc[index_n, 'restaurant_id']

# url = url_df.loc[index_n, 'restaurant_url']

# # id = 162938

# print(id)
# print(url)


# cmt_data = []

# params['ResId'] = id
# params['ExcludeIds'] = ''

# print(params)

# re = requests.get(root_url, params = params, headers = headers)
# print(re.url)
# content = re.json()
# cmts = content['Items']

# counter = 0
# old_cmt_num = 0

# while len(cmts):
#     print('counter:  ', counter)
#     print('len cmts: ', len(cmts))
#     counter+=1

#     tempdf = pd.json_normalize(cmts)
#     cmt_data.append(tempdf)

#     temp_excludeIds = tempdf.Id.tolist()
#     params['ExcludeIds'] += ','+ str(temp_excludeIds).replace(']','').replace('[','')
#     params['ExcludeIds'] = params['ExcludeIds'].strip(',')


#     temp_ls = params['ExcludeIds']
#     temp_list = list(set(temp_ls.split(',')))
#     params['ExcludeIds'] = ','.join(temp_list).strip(',')

#     # print(params['ExcludeIds'])
#     print('len params:   ',len(temp_list))

#     new_cmt_num = len(temp_list)
    
#     if old_cmt_num == new_cmt_num:
#         break
#     else:
#         old_cmt_num = new_cmt_num

#     re = requests.get(root_url, params = params, headers = headers)
#     cmts = re.json()['Items']
#     # print('len cmts:   ', len(cmts))



# print("len cmt_data: ", len(cmt_data))

# if len(cmt_data):
#     data = pd.concat(cmt_data, ignore_index = True)
#     data['Restaurant_id'] = content['ResId']
#     print("enter :sldfksdjlf ")
# print("end")

In [ ]:
# data

In [ ]:
# data = []
# ex_ls = []
# total_except = 0

# max_workers = 1000

# ii = 0
# num = 5000

# begin = num* ii
# length1 = begin + num
# length2 = len(url_df)
# length = min(length1, length2)

# pbar = tqdm(total = (length - begin))


# with concurrent.futures.ThreadPoolExecutor(max_workers= max_workers) as executor:

#     # for i in tqdm(range(begin, length, max_workers)):
#     index = begin
    
#     while index < length:
#         i = index 
#         future_samples = []

#         for index in range(max_workers):
#             id = url_df.loc[index + i, 'restaurant_id']
#             sample = executor.submit(crawl_cmt2, id , params.copy())
#             future_samples.append(sample)
#             index += 1
#             if index >= length:
#                 break

#         concurrent.futures.wait(future_samples)

#         for j in range(len(future_samples)):
#             try:
#                 tempdata = future_samples[j].result()
#                 if len(tempdata):
#                     data.append(tempdata)
#             except Exception as exc:
#                 total_except += 1
#                 ex_ls.append(i+j)
#                 print('Generated an exception: %s' % (exc))
#                 print('index: ', i+j)
#                 print()
#             pbar.update(1)

# print("total except:   ", total_except)



#### save data

In [ ]:
len(data)

1226

In [ ]:
data_cmt = pd.concat(data, ignore_index=True)
data_cmt.Id.nunique()

15187

In [ ]:
data_cmt

,Id,Title,Description,Type,TypeName,CreatedDate,CreatedOnTimeDiff,DeviceName,DeviceUrl,DeviceType,...,Owner.MobileCover,Options.Guest,Options.VisitAgain,Options.MoneySpend,Restaurant_id,Video.Id,Video.Source,Video.Avatar,Video.Type,Video.SourceType
0,10293551,10 điểm cho chất lượng,Mình order 2 bịch bún thái của quán\nQuán rất ...,1,đã bình luận,/Date(1608729801790)/,23/12/2020,iPhone,/ung-dung-mobile,1,...,None,NaN,NaN,NaN,1057030,NaN,NaN,NaN,NaN,NaN
1,9966140,Bún Bò & Bún Thái 357,Ai là fan của bún bò hay bún thái thì không th...,1,đã bình luận,/Date(1607497359267)/,09/12/2020,Android,/ung-dung-mobile,2,...,None,NaN,NaN,NaN,1057030,NaN,NaN,NaN,NaN,NaN
2,9877008,Yummy yummy,Thèm bún thái lên foody order đại quán gần nhà...,1,đã bình luận,/Date(1607155683417)/,05/12/2020,iPhone,/ung-dung-mobile,1,...,None,NaN,Chắc chắn,NaN,1057030,NaN,NaN,NaN,NaN,NaN
3,9837273,Đáng để thử,"Bún thái nước dùng đậm đà, giá cả hợp lý, sẽ q...",1,đã bình luận,/Date(1606999277753)/,03/12/2020,iPhone,/ung-dung-mobile,1,...,None,NaN,Chắc chắn,NaN,1057030,NaN,NaN,NaN,NaN,NaN
4,9815583,Buổi sáng vui vẻ,"Bún bò ngon thịt giò chả cua, giao hàng nhanh,...",1,đã bình luận,/Date(1606961673980)/,03/12/2020,iPhone,/ung-dung-mobile,1,...,None,NaN,Chắc chắn,NaN,1057030,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16131,1138638,Papa's Chicken & Beer Thủ Đức,"Mình gọi món gà cay phô mai, ăn khá ngon....",1,đã bình luận,/Date(1479629812013)/,20/11/2016,Android,/ung-dung-mobile,2,...,None,NaN,NaN,NaN,71431,NaN,NaN,NaN,NaN,NaN
16132,1083454,Cũng OK,Mình và bạn tới đây ăn lần đầu vào một tối. \n...,1,đã bình luận,/Date(1476525505993)/,15/10/2016,Web,None,4,...,None,2+,Có thể,"200,000đ +",71431,NaN,NaN,NaN,NaN,NaN
16133,1040920,Papa's Chicken & Beer Thủ Đức,Hôm CN mình có ghé quán. Đây là lần đầu...,1,đã bình luận,/Date(1473759761587)/,13/09/2016,Android,/ung-dung-mobile,2,...,None,NaN,NaN,NaN,71431,NaN,NaN,NaN,NaN,NaN
16134,1033116,Sẽ ghé nhiều lần,Món ăn không đa dạng nhưng món nào cũng ngon h...,1,đã bình luận,/Date(1473249155927)/,07/09/2016,iPhone,/ung-dung-mobile,1,...,None,2+,Chắc chắn,"100,000đ +",71431,NaN,NaN,NaN,NaN,NaN


In [ ]:
data_cmt.Restaurant_id.nunique()

1226

In [ ]:
data_cmt.to_csv(f'drive/MyDrive/Data/cmt_data1/cmt_data{ii}.csv')

100%|██████████| 4000/4000 [03:20<00:00, 65.50it/s]

In [ ]:
fail_ids

[]

In [ ]:
fail_df = pd.DataFrame({'id': fail_ids})
fail_df

,id


In [ ]:
if len(fail_ids):
    fail_df.to_csv(f'drive/MyDrive/Data/cmt_data1/fail_ids{ii}.csv')
# if ip is not block, the if clause should return 0 and file will not be save.

## Concat all the result files to a big file.